In [5]:
import os, datetime
from google.colab import files
import shutil, os, datetime
import os, glob, datetime
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [6]:

hoje = datetime.date.today()
BASE = "/content/projeto_pcb"
BRONZE = f"{BASE}/dados/bronze/ano={hoje.year}/mes={hoje:%m}"
SILVER = f"{BASE}/dados/silver/ano={hoje.year}/mes={hoje:%m}"
GOLD   = f"{BASE}/dados/gold"

for d in [BRONZE, SILVER, GOLD]:
    os.makedirs(d, exist_ok=True)

BASE, BRONZE, SILVER, GOLD


('/content/projeto_pcb',
 '/content/projeto_pcb/dados/bronze/ano=2025/mes=10',
 '/content/projeto_pcb/dados/silver/ano=2025/mes=10',
 '/content/projeto_pcb/dados/gold')

In [7]:
arquivos = [f for f in os.listdir("/content") if f.endswith(".csv")]

if arquivos:
    nome_arquivo = arquivos[0]  # pega o primeiro CSV encontrado
    origem = f"/content/{nome_arquivo}"
    destino = os.path.join(
        BRONZE,
        f"{os.path.splitext(nome_arquivo)[0]}_{hoje}.csv"
    )

    shutil.move(origem, destino)
    print(f"✅ Arquivo movido para Bronze: {destino}")

    df = pd.read_csv(destino)
    print(f"\nPrévia dos dados ({len(df)} linhas):")
    display(df.head())


# up = files.upload()
# nome_arquivo = list(up.keys())[0]
# dest = os.path.join(BRONZE, f"{os.path.splitext(nome_arquivo)[0]}_{datetime.date.today()}.csv")
# shutil.move('focos_mensal_br_202301', dest)
# print("Salvo em bronze:", dest)
l

✅ Arquivo movido para Bronze: /content/projeto_pcb/dados/bronze/ano=2025/mes=10/focos_mensal_br_202403_2025-10-09.csv

Prévia dos dados (6636 linhas):


,id,lat,lon,data_hora_gmt,satelite,municipio,estado,pais,municipio_id,estado_id,pais_id,numero_dias_sem_chuva,precipitacao,risco_fogo,bioma,frp
0,5a21ec08-77bf-368c-915b-dc7a2bfbbe1a,1.5951,-60.5378,2024-03-01 00:04:57,GOES-16,CARACARAÍ,RORAIMA,Brasil,1400209,14,33.0,10.0,0.0,1.0,Amazônia,79.0
1,c05c52f7-ed51-3445-9a7b-5e5ba65622b2,1.5951,-60.5568,2024-03-01 00:04:57,GOES-16,CARACARAÍ,RORAIMA,Brasil,1400209,14,33.0,10.0,0.0,1.0,Amazônia,83.0
2,6198aafd-2cbd-3817-b173-990013d9092f,1.5586,-60.5571,2024-03-01 00:04:57,GOES-16,CARACARAÍ,RORAIMA,Brasil,1400209,14,33.0,14.0,0.0,1.0,Amazônia,65.6
3,0b71d402-9f3b-324a-b82d-2906cbfadcf4,1.5769,-60.5380,2024-03-01 00:04:57,GOES-16,CARACARAÍ,RORAIMA,Brasil,1400209,14,33.0,12.0,0.0,1.0,Amazônia,60.4
4,4624ba4a-1de3-3a01-92d5-6e008df2f2a3,1.5768,-60.5569,2024-03-01 00:04:57,GOES-16,CARACARAÍ,RORAIMA,Brasil,1400209,14,33.0,12.0,0.0,1.0,Amazônia,64.3


In [8]:
# paths = sorted(glob.glob(os.path.join(BRONZE, "*.csv")))
# assert paths, "Nenhum CSV encontrado em bronze."
# df = pd.concat((pd.read_csv(p) for p in paths), ignore_index=True)

# esperadas = {"X","Y","R","G","B"}
# assert esperadas.issubset(set(df.columns)), f"Faltam colunas: {esperadas - set(df.columns)}"
# if "Grey" not in df.columns:
#     df["Grey"] = 0  # se não existir, inicializa


# for c in ["X","Y","R","G","B","Grey"]:
#     df[c] = pd.to_numeric(df[c], errors="coerce")
# df = df.dropna(subset=["X","Y","R","G","B"]).copy()


# df["X"] = df["X"].round().astype("Int64")
# df["Y"] = df["Y"].round().astype("Int64")
# for c in ["R","G","B"]:
#     df[c] = df[c].clip(0,255).round().astype(np.uint8)


# df["Grey"] = df["Grey"].fillna(0).clip(0,1).astype(int)


# df["dt_carga"] = datetime.date.today().isoformat()

# # Salva silver
# silver_csv = os.path.join(SILVER, f"pcb_xyrgb_silver_{datetime.date.today()}.csv")
# df.to_csv(silver_csv, index=False)
# print("Silver salvo:", silver_csv, "| linhas:", len(df))



In [9]:
import os, glob, datetime
import pandas as pd
import numpy as np

paths = sorted(glob.glob(os.path.join(BRONZE, "*.csv")))
assert paths, "Nenhum CSV encontrado em bronze."

#LEMBRAR dessa linha mais na frente quando tiver mais meses / tratar isso no bronze
df = pd.concat((pd.read_csv(p) for p in paths), ignore_index=True)

esperadas = {"id","lat","lon","data_hora_gmt","satelite","municipio","estado","pais"}
faltando = esperadas - set(df.columns)
assert not faltando, f"Faltam colunas no CSV: {faltando}"

# id como texto (alguns ids têm hífens/UUID)
df["id"] = df["id"].astype(str).str.strip()


df["lat"] = pd.to_numeric(df["lat"], errors="coerce")
df["lon"] = pd.to_numeric(df["lon"], errors="coerce")

df["data_hora_gmt"] = pd.to_datetime(df["data_hora_gmt"], errors="coerce", utc=True)

for c in ["satelite","municipio","estado","pais"]:
    df[c] = df[c].astype(str).str.strip()

for c in ["municipio_id","estado_id","pais_id"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").astype("Int64")

df = df.dropna(subset=["lat","lon","data_hora_gmt"]).copy()

#desfazer
df["lat"] = df["lat"].round(6)
df["lon"] = df["lon"].round(6)

df["ano"] = df["data_hora_gmt"].dt.year.astype("Int64")
df["mes"] = df["data_hora_gmt"].dt.month.astype("Int64")
df["dia"] = df["data_hora_gmt"].dt.day.astype("Int64")

# carga que prof pediu
df["dt_carga"] = datetime.date.today().isoformat()

# Salva silver
silver_csv = os.path.join(SILVER, f"focos_silver_{datetime.date.today():%Y%m%d}.csv")
os.makedirs(SILVER, exist_ok=True)
df.to_csv(silver_csv, index=False)

print("Silver salvo:", silver_csv, "| linhas:", len(df))
df.head()


Silver salvo: /content/projeto_pcb/dados/silver/ano=2025/mes=10/focos_silver_20251009.csv | linhas: 6636


,id,lat,lon,data_hora_gmt,satelite,municipio,estado,pais,municipio_id,estado_id,pais_id,numero_dias_sem_chuva,precipitacao,risco_fogo,bioma,frp,ano,mes,dia,dt_carga
0,5a21ec08-77bf-368c-915b-dc7a2bfbbe1a,1.5951,-60.5378,2024-03-01 00:04:57+00:00,GOES-16,CARACARAÍ,RORAIMA,Brasil,1400209,14,33,10.0,0.0,1.0,Amazônia,79.0,2024,3,1,2025-10-09
1,c05c52f7-ed51-3445-9a7b-5e5ba65622b2,1.5951,-60.5568,2024-03-01 00:04:57+00:00,GOES-16,CARACARAÍ,RORAIMA,Brasil,1400209,14,33,10.0,0.0,1.0,Amazônia,83.0,2024,3,1,2025-10-09
2,6198aafd-2cbd-3817-b173-990013d9092f,1.5586,-60.5571,2024-03-01 00:04:57+00:00,GOES-16,CARACARAÍ,RORAIMA,Brasil,1400209,14,33,14.0,0.0,1.0,Amazônia,65.6,2024,3,1,2025-10-09
3,0b71d402-9f3b-324a-b82d-2906cbfadcf4,1.5769,-60.5380,2024-03-01 00:04:57+00:00,GOES-16,CARACARAÍ,RORAIMA,Brasil,1400209,14,33,12.0,0.0,1.0,Amazônia,60.4,2024,3,1,2025-10-09
4,4624ba4a-1de3-3a01-92d5-6e008df2f2a3,1.5768,-60.5569,2024-03-01 00:04:57+00:00,GOES-16,CARACARAÍ,RORAIMA,Brasil,1400209,14,33,12.0,0.0,1.0,Amazônia,64.3,2024,3,1,2025-10-09


In [10]:
padrao_silver = "focos_silver_*.csv"   # ajuste se usou outro nome

paths = sorted(glob.glob(os.path.join(SILVER, padrao_silver)))
assert paths, f"Nenhum CSV Silver encontrado em: {SILVER}"

# parse_dates  reforçandos a conversão em seguida
df = pd.concat((pd.read_csv(p, parse_dates=["data_hora_gmt"]) for p in paths),
               ignore_index=True)

# Garante que data_hora_gmt é datetime
df["data_hora_gmt"] = pd.to_datetime(df["data_hora_gmt"], errors="coerce", utc=True)

# Normalizações úteis para agregação
df["data"] = df["data_hora_gmt"].dt.date
df["ano"]  = df["data_hora_gmt"].dt.year
df["mes"]  = df["data_hora_gmt"].dt.month
df["hora"] = df["data_hora_gmt"].dt.hour

por_estado_mes = (
    df.groupby(["estado","ano","mes"])
      .size()
      .reset_index(name="qtd_focos")
      .sort_values(["ano","mes","qtd_focos"], ascending=[True,True,False])
)


por_municipio_dia = (
    df.groupby(["estado","municipio","data"])
      .size()
      .reset_index(name="qtd_focos")
      .sort_values(["data","qtd_focos"], ascending=[True,False])
)


por_satelite_mes = (
    df.groupby(["satelite","ano","mes"])
      .size()
      .reset_index(name="qtd_focos")
      .sort_values(["ano","mes","qtd_focos"], ascending=[True,True,False])
)

por_satelite_total = (
    df.groupby("satelite")
      .size()
      .reset_index(name="qtd_focos")
      .assign(perc=lambda x: (x["qtd_focos"] / x["qtd_focos"].sum())*100)
      .sort_values("qtd_focos", ascending=False)
      .round({"perc": 2})
)

df["lat_cell"] = df["lat"].round(2)
df["lon_cell"] = df["lon"].round(2)
grade_espacial = (
    df.groupby(["lat_cell","lon_cell"])
      .size()
      .reset_index(name="qtd_focos")
      .sort_values("qtd_focos", ascending=False)
)

# ------------------------------
# Salva em GOLD
# ------------------------------
os.makedirs(GOLD, exist_ok=True)
hoje = datetime.date.today().isoformat()

g1 = os.path.join(GOLD, f"gold_focos_por_estado_mes_{hoje}.csv")
g2 = os.path.join(GOLD, f"gold_focos_por_municipio_dia_{hoje}.csv")
g3 = os.path.join(GOLD, f"gold_focos_por_satelite_mes_{hoje}.csv")
g3b= os.path.join(GOLD, f"gold_focos_por_satelite_total_{hoje}.csv")
g4 = os.path.join(GOLD, f"gold_grade_espacial_{hoje}.csv")

por_estado_mes.to_csv(g1, index=False)
por_municipio_dia.to_csv(g2, index=False)
por_satelite_mes.to_csv(g3, index=False)
por_satelite_total.to_csv(g3b, index=False)
grade_espacial.to_csv(g4, index=False)

print("Gold salvo:")
print(" -", g1)
print(" -", g2)
print(" -", g3)
print(" -", g3b)
print(" -", g4)

try:
    ult_ano  = por_estado_mes["ano"].max()
    ult_mes  = por_estado_mes.query("ano == @ult_ano")["mes"].max()
    top_estados = (
        por_estado_mes
        .query("ano == @ult_ano and mes == @ult_mes")
        .nlargest(10, "qtd_focos")
    )
    print(f"\nTOP estados no mês mais recente ({ult_ano}-{ult_mes:02d}):")
    print(top_estados.to_string(index=False))
except Exception as e:
    print("\n[Aviso] Não foi possível imprimir TOP estados do mês mais recente:", e)

print("\n% por satélite (total):")
print(por_satelite_total.to_string(index=False))

Gold salvo:
 - /content/projeto_pcb/dados/gold/gold_focos_por_estado_mes_2025-10-09.csv
 - /content/projeto_pcb/dados/gold/gold_focos_por_municipio_dia_2025-10-09.csv
 - /content/projeto_pcb/dados/gold/gold_focos_por_satelite_mes_2025-10-09.csv
 - /content/projeto_pcb/dados/gold/gold_focos_por_satelite_total_2025-10-09.csv
 - /content/projeto_pcb/dados/gold/gold_grade_espacial_2025-10-09.csv

TOP estados no mês mais recente (2024-03):
            estado  ano  mes  qtd_focos
           RORAIMA 2024    3       1767
       MATO GROSSO 2024    3       1508
             BAHIA 2024    3        936
MATO GROSSO DO SUL 2024    3        562
             GOIÁS 2024    3        429
         SÃO PAULO 2024    3        282
      MINAS GERAIS 2024    3        260
         TOCANTINS 2024    3        132
              PARÁ 2024    3         98
           ALAGOAS 2024    3         93

% por satélite (total):
 satelite  qtd_focos  perc
  NOAA-20       1870 28.18
  NPP-375       1399 21.08
 NPP-375D      

In [11]:
# cont = df["Grey"].value_counts().sort_index()
# plt.bar(cont.index.astype(str), cont.values)
# plt.title("Distribuição da classe Grey")
# plt.xlabel("Grey (0=não cinza, 1=cinza)")
# # plt.ylabel("n")
# plt.show()


In [12]:
# agg = df.groupby("Grey")[["R","G","B"]].mean().reset_index()
# for ch in ["R","G","B"]:
#     plt.bar(agg["Grey"].astype(str), agg[ch], label=ch)
#     plt.title(f"Média do canal {ch} por Grey")
#     plt.xlabel("Grey")
#     plt.ylabel(f"Média {ch} (escala {'0–255' if df['R'].max()>1 else '0–1'})")
#     plt.show()
